> # **Data Preprocessing/Procesamiento de los Datos**

Para aplicar una Convolutional Neural Network (CNN) en deep learning, los datos X e Y deben cumplir con ciertos requisitos y características.

***Datos de entrada (X)***:
1. ***Dimensiones de los datos de entrada***: Los datos de entrada X deben tener una estructura tridimensional, donde la primera dimensión represente el número de muestras (instancias), la segunda dimensión represente la altura de la imagen, y la tercera dimensión represente la anchura de la imagen.

2. ***Normalización de los datos***: Es importante normalizar los datos de entrada para que estén en un rango común y para ayudar al algoritmo a converger más rápidamente. Normalmente, los datos de imagen se normalizan dividiendo cada valor de píxel por 255 (el valor máximo de un canal de color de 8 bits).

3. ***Preprocesamiento de los datos***: redimensionar las imágenes a un tamaño específico, recortarlas, cambiar su orientación, entre otros, para asegurar que todas las muestras tengan la misma forma.

***Datos de salida (y)***:
1. ***Codificación de etiquetas***: Las etiquetas y deben estar codificadas como números enteros que representen las clases. Esto se hace utilizando la codificación one-hot, donde cada clase se representa como un vector binario con un 1 en la posición correspondiente a la clase y 0 en todas las demás posiciones.

2. ***Conjunto de datos balanceado***: Es importante que el conjunto de datos esté balanceado, lo que significa que hay aproximadamente la misma cantidad de muestras en cada clase. Esto ayuda a evitar sesgos en el modelo hacia las clases más comunes.




En esta etapa, realizamos las transformaciones necesarias para que nuestros datos estén adecuadamente preparados para la fase de modelado:

**1. Creacion X, y()**: Reshape: (256, 256, 3) <br>
**2. Data preprocessing(X,y)**:
  * estandarizacion/ escalado de pixeles
  * pasar a numerico la v target

**3. Data Augmentation**

## **Librerias**

In [ ]:
# Data Analysis
import os
import numpy as np
import pandas as pd
import pickle

# Data Preproccesing
from matplotlib.image import imread
from PIL import Image
import cv2
from skimage.transform import resize
from tensorflow.keras.utils import to_categorical

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Control de versiones
print('Numpy version: {}'.format(np.__version__))
print('Pandas version: {}'.format(pd.__version__))
print('Seaborn version: {}'.format(sns.__version__))

Numpy version: 1.26.4
Pandas version: 2.2.2
Seaborn version: 0.13.2


## **Rutas**

In [ ]:
# Rutas de cada dataset: train-test-val:
train_path = "/content/drive/MyDrive/Colab Notebooks/MAIZ/src/data/data_splitted/train"
test_path = "/content/drive/MyDrive/Colab Notebooks/MAIZ/src/data/data_splitted/test"
val_path = "/content/drive/MyDrive/Colab Notebooks/MAIZ/src/data/data_splitted/val"

In [ ]:
# Clases
clases = os.listdir("/content/drive/MyDrive/Colab Notebooks/MAIZ/src/data/data_splitted/train")
clases

['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

## **1. Creación de variables: X, y**
* X_train, y_train
* X_test, y_test
* X_val, y_val


In [ ]:
def creacion_listas_X_y(folder_path):
  X = []
  y = []
  #Lista con los nombres de cada clase:
  clases = os.listdir("/content/drive/MyDrive/Colab Notebooks/MAIZ/src/data/data_splitted/train")
  # Definimos las dimensiones de las imágenes
  img_width, img_height = 256, 256 # image_size

  # Iteramos en clases
  for class_name in clases: # lista nombres clases
    # Lista de las imagenes
    images_names = os.listdir(folder_path +'/'+ class_name)
    # Iteramos en las imagenes
    for img_name in images_names:
      image_path = folder_path +'/'+ class_name +'/'+ img_name # ruta de la imagen
      image_array = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB) # pasamos la imagen a array con la imagen a color
      image = cv2.resize(image_array, (img_width, img_height)) # data preprocessing
      image = np.array(image) # usamos numpy para tener la imagen en array
      image = image.astype('float32') # pasamos la imagen a tipo 'float32'
      X.append(image) # agregamos la imagen a X
      y.append(class_name) # agregamos su nombre/target a y

  return X, y # retornamos una tupla con los valores (X, y)

In [ ]:
# Creamos X, y:
X_train, y_train = creacion_listas_X_y(train_path)
X_test, y_test = creacion_listas_X_y(test_path)
X_val, y_val = creacion_listas_X_y(val_path)

## **2. Data Transformation**



### **X**<br>

Aplicaremos las siguientes Transformaciones:

**1. Estandarizacion de los Datos**:
* Los valores del canal RGB (píxeles) están en el rango [0, 255].
* Normalizamos los valores para que estén en el rango [0, 1].


In [ ]:
def transformacion_datos_X(X_train, X_test, X_val):

  # Convertimos las listas en arrays:
  X_train = np.array(X_train)
  X_test = np.array(X_test)
  X_val = np.array(X_val)

  # Creamos una lista con las X:
  data_X = [X_train, X_test, X_val]
  for X in data_X:
    # Normalizamos las imagenes a valores entre [0, 1]:
    for i in range(0, len(X)):
      X[i] /= 255

  return X_train, X_test, X_val

In [ ]:
# Procesamos X_train, X_test, X_val:
X_train, X_test, X_val = transformacion_datos_X(X_train, X_test, X_val)

In [ ]:
# Visualizamos las dimensiones
print("X_train: {}".format(X_train.shape))
print("X_test: {}".format(X_test.shape))
print("X_val: {}".format(X_val.shape))

X_train: (3348, 256, 256, 3)
X_test: (423, 256, 256, 3)
X_val: (417, 256, 256, 3)


In [ ]:
# Visualizamos Normalizacion
print("Escalado: [{}, {}]".format(np.min(X_train[0]), np.max(X_train[0])))

Escalado: [0.0, 1.0]


In [ ]:
# copia de las targets para no estar ejecutando todo nuevamente:
y_train_copy = np.copy(y_train)
y_test_copy = np.copy(y_test)
y_val_copy = np.copy(y_val)

### **Target : y**

Debido a que nuestro target original está en formato categórico, deberemos transformalo a numérico para poder aplicar posteriormente modelos de Deep Learning y Transfer Learning.




In [ ]:
# Visualizemos los primeros datos..
y_train[0:12]

['Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight',
 'Blight']

#### **Categórica a Numérica**

***1. One Hot Encoding of Labels***

* El objetivo de esta accion es convertir los datos de la etiqueta de clase al mismo formato de la salida de nuestra red neuronal,como vectores numéricos binarios one-hot , en el que solo un elemento es 1 (indicando la categoría activa) y los demás son 0.

* En nuestro caso:

  Si `y = 'Blight'` --> `y = [[1,0,0,0]]`<br>
  Si `y = 'Common_Rust'` --> `y = [[0,1,0,0]]`<br>
  Si `y = 'Gray_Leaf_Spot'` --> `y = [[0,0,1,0]]`<br>
  Si `y = 'Healthy'` --> `y = [[0,0,0,1]]`


  Usando **TensorFlow**:
  * Aplicando la función `tf.keras.utils.to_categorical`
  * Usando **diccionario y mapeando**:

In [ ]:
def transformacion_datos_y_OneHotEncoding(y_train, y_test, y_val):
  # 1°) Lista con los nombres de las clases:
  class_names = clases

  # 2°) Creamos diccionario: {clase1:0,...., claseN:n}
  clases_a_num = {class_name : index for index, class_name in enumerate(class_names)}

  # 3°) Mapeo
  y_train = list(map(lambda etiqueta: clases_a_num[etiqueta], y_train)) # etiqueta=label=numero [0,1,2,3]
  y_test = list(map(lambda etiqueta: clases_a_num[etiqueta], y_test))
  y_val = list(map(lambda etiqueta: clases_a_num[etiqueta], y_val))
  '''
  Argumentos de la funcion lambda :
  qué es lo que queremos que nos devuelva : condición/lo que buscamos

  Argumentos de la funcion Map(funcion, iterable)
  * Funcion: Es una función que se aplicará a cada elemento del iterable, en nuestro caso usamos lambda
  * iterable: Es la secuencia de elementos a la que se aplicará la función. Ej:lista, tupla

  list(): map() devuelve un objeto iterable, no puedes acceder a sus elementos directamente o inspeccionarlo fácilmente
          Por eso convertimos el resultado de map en una lista iterable
  '''

  # 4°) Conversion de v targets a numéricas:
  y_train = to_categorical(y_train,
                           num_classes=4)

  y_test = to_categorical(y_test,
                          num_classes=4)

  y_val = to_categorical(y_val,
                         num_classes=4)

  return  y_train, y_test, y_val

In [ ]:
# Procesamos y_train, y_test, y_val:
y_train_ohe, y_test_ohe, y_val_ohe = transformacion_datos_y_OneHotEncoding(y_train, y_test, y_val)

In [ ]:
# Visualizamos las dimensiones
print("y_train: {}".format(y_train_ohe.shape))
print("y_test: {}".format(y_test_ohe.shape))
print("y_val: {}".format(y_val_ohe.shape))

y_train: (3348, 4)
y_test: (423, 4)
y_val: (417, 4)


In [ ]:
# Visualizamos la transformacion:
y_train_ohe[0:12]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [ ]:
y_test_ohe[0:12]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [ ]:
y_val_ohe[:12]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

***2. Label Encoding***

* Asignamos un número entero único a cada categoría

* En nuestro caso:

`{'Blight':0, 'Common_Rust':1,'Gray_Leaf_Spot':2, 'Healthy':3}`


In [ ]:
def transformacion_datos_y_LabelEncoding(y_train, y_test, y_val):
  # Conversion de v targets a numéricas
  targets = [y_train, y_test, y_val]
  # Crear diccionario con los nombres de las clases y sus valores numericos
  d = {'Blight':0, 'Common_Rust':1, 'Gray_Leaf_Spot':2, 'Healthy':3}

  # Iteramos en train-test-val:
  for y in targets:
    # Itermaos en y
    for i, clase in enumerate(y): #enumerate() devuelve una tupla que contiene el índice de la clase en la lista y la clase en sí misma
      y[i] = d[clase]

  # Transformamos: en numpy arrays, tipo float y redimensionamos:
  y_train = np.asarray(y_train).astype('float32').reshape((len(y_train),1))
  y_test = np.asarray(y_test).astype('float32').reshape((len(y_test),1))
  y_val = np.asarray(y_val).astype('float32').reshape((len(y_val),1))

  # Como tenemos un array 2D, lo aplanamos para que tenga la siguiente dimension: `(n_samples,)`
  y_train = y_train.ravel()
  y_test = y_test.ravel()
  y_val = y_val.ravel()

  return  y_train, y_test, y_val

In [ ]:
# Procesamos y_train, y_test, y_val:
y_train_le, y_test_le, y_val_le = transformacion_datos_y_LabelEncoding(y_train, y_test, y_val)

In [ ]:
# Visualizamos las dimensiones
print("y_train: {}".format(y_train_le.shape))
print("y_test: {}".format(y_test_le.shape))
print("y_val: {}".format(y_val_le.shape))

y_train: (3348,)
y_test: (423,)
y_val: (417,)


In [ ]:
# Visualizamos la codificacion:
print("y_train: {}".format(y_train[:12]))
print("y_test: {}".format(y_test[:12]))
print("y_val: {}".format(y_val[:12]))

y_train: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_test: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_val: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
